In [29]:
from keras.backend import conv2d
from keras.layers.merge import add
from keras.layers.pooling import MaxPooling2D
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPool2D 
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [30]:
# fetching the training data set from local directory

trainDirectory = r"../input/face-expression-recognition-dataset/images/train"
validationDirectory = r"../input/face-expression-recognition-dataset/images/validation"

train_data = ImageDataGenerator()
validation_data = ImageDataGenerator()

training_list = train_data.flow_from_directory(directory=trainDirectory,target_size=(48,48),color_mode="grayscale",batch_size=100,shuffle=True,class_mode='categorical')
validation_list = validation_data.flow_from_directory(directory=validationDirectory,target_size=(48,48),color_mode='grayscale',batch_size=100,shuffle=True,class_mode='categorical')

In [31]:
#neural networks
no_of_classes = 7

model = Sequential()

model.add(Conv2D(128,(3,3),input_shape = (48,48,1),activation="relu"))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Conv2D(512,(3,3),activation="relu"))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Conv2D(256,(3,3),activation="relu"))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Conv2D(128,(3,3),activation="relu"))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Flatten())

model.add(Dense(512,activation="relu"))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(256,activation="relu"))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(no_of_classes, activation='softmax'))

In [32]:
filePath = './saved_model/model-{epoch:03d}.h5'
checkpoint = ModelCheckpoint(filepath=filePath, monitor='val_acc', verbose=1, save_best_only=False, mode='max')

early_stopping = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          )

reduce_learningrate = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks_list = [early_stopping,checkpoint,reduce_learningrate]

model.compile(
              loss="CategoricalCrossentropy",
              optimizer="adam",
              metrics=['accuracy'])
model.summary()

history = model.fit_generator(generator=training_list,
                                steps_per_epoch=training_list.n//training_list.batch_size,
                                epochs=50,
                                validation_data = validation_list,
                                validation_steps = validation_list.n//validation_list.batch_size,
                                callbacks=callbacks_list
                                )